In [1]:
%run setup.py

from configs import utils
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pl.Config.set_tbl_hide_column_data_types(True)
pl.Config.set_tbl_cols(-1)

lf_dog_move_data = utils.load_large_data("../data/processed", "DogMoveData_Clean.csv", schema=utils.dog_move_data_schema)

In [2]:
dog_16 = lf_dog_move_data.filter(pl.col("DogID") == 16)

dog_16_3min = dog_16.filter(
    pl.col("t_sec").is_between(90.81, 90.81 + 3 * 60)
).collect()

In [ ]:
df_pd_16 = dog_16_3min.to_pandas()

plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=df_pd_16, 
    x="t_sec", 
    y="ABack_x", 
    hue="Behavior_1", 
    s=10,
    alpha=0.7,
    edgecolor='none'
)
plt.title("Aceleração do eixo X das costas ao Longo do Tempo")
plt.xlabel("Tempo (s)")
plt.ylabel("Aceleração do eixo X (m/s²)")
plt.legend(title="Comportamento", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("../reports/figures/aceleracao_eixo_x_das_costas.png", dpi=300)
plt.show()

In [11]:
dog_61 = lf_dog_move_data.filter(pl.col("DogID") == 61)

dog_61_3min = dog_61.filter(
    pl.col("t_sec").is_between(1000, 1000 + 3 * 60)
).collect()

In [ ]:
df_pd_61 = dog_61_3min.to_pandas()

plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=df_pd_61, 
    x="t_sec", 
    y="ANeck_x",
    hue="Behavior_1",
    s=10,
    alpha=0.7,
    edgecolor=None
)
plt.title("Aceleração do eixo X do pescoço ao Longo do Tempo")
plt.xlabel("Tempo (s)")
plt.ylabel("Aceleração (m/s²)")
plt.legend(title="Comportamento", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("../reports/figures/aceleracao_eixo_x_do_pescoco.png", dpi=300)
plt.show()

In [21]:
filtered_lf = lf_dog_move_data.filter(
    pl.col("Behavior_1")
        .is_in(["Lying chest", "Standing", "Sitting"]))

sample = filtered_lf.collect().sample(n=100_000, seed=42)

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=sample,
    x="Behavior_1",
    y="ABack_y",
    hue="Behavior_1"
)
plt.title("Boxplot da Aceleração do eixo Y das costas por Comportamento")
plt.xlabel("Comportamento")
plt.ylabel("Aceleração (m/s²)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("../reports/figures/boxplot_aceleracao_eixo_y_das_costas.png", dpi=300)
plt.show()

In [ ]:
sensor_cols = [
    "ABack_x", "ABack_y", "ABack_z",
    "ANeck_x", "ANeck_y", "ANeck_z",
    "GBack_x", "GBack_y", "GBack_z",
    "GNeck_x", "GNeck_y", "GNeck_z",
]

sample_sensors_pd = sample.select(sensor_cols).to_pandas()

plt.figure(figsize=(9, 7))
corr = sample_sensors_pd.corr(numeric_only=True)
ax = sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    fmt=".2f",
    xticklabels=corr.columns,
    yticklabels=corr.index,
    square=True,
    cbar_kws={"shrink": 0.8},
)
plt.title("Correlação entre sinais dos sensores")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("../reports/figures/heatmap_correlacao_sensores.png", dpi=300)
plt.show()